# Blackjack with AI player

In [83]:
import random

In [84]:
RANKS = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Card class keeps:
    # rank - The rank of the card [2,3,4,5,6,7,8,9,10,J,Q,K,A]
    # value - The point value associated with the rank
    # isAce - True if the card is an Ace
class Card(object):
    
    def __init__(self, rank = None):
        self.rank = rank
        
        if not rank: # null card
            self.value = 0
            self.isAce = False
            return
        elif rank == 'A':
            self.value = 11
            self.isAce = True
        elif rank in ['J','Q','K']:
            self.value = 10
            self.isAce = False
        else:
            self.value = int(rank) # 2-10
            self.isAce = False
            
    def __str__(self):
        return self.rank
    
    def __eq__(self, card):
        return self.value == card.value
    
    def __repr__(self):
        return self.rank


# Deck class keeps:
    # cards - list of cards remaining in the deck
class Deck(object):
    
    def __init__(self, nDecks = 1):
        self.cards = []
        self.buildDeck(nDecks)
        self.prepareDeck(nDecks)
    
    def buildDeck(self, nDecks):
        for i in range(nDecks):
            for j in range(4): # 4 of each card per deck
                for rank in RANKS:
                    self.cards.append(Card(rank))
    
    def prepareDeck(self, nDecks):
        for i in range(7*nDecks):
            self.shuffle()
            
    def shuffle(self):
        write_index = 0 # to write,
        while write_index < len(self.cards):
            read_index = random.randint(write_index, len(self.cards)-1)
            self.cards[read_index], self.cards[write_index] = self.cards[write_index], self.cards[read_index]
            write_index += 1
        
    def draw(self):
        card = self.cards.pop(0)
        return card
    
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
    
    
# Hand class keeps
    # hand - cards in the hand
    # bet - the bet placed on the hand
class Hand(object):
    
    def __init__(self, card = None, bet = 50):
        self.bet = bet
        self.cards = []
        if card:
            self.cards.append(card)
        
    def addCard(self, card):
        self.cards.append(card)     
            
    def removeCard(self):
        return self.cards.pop()
    
    def isBust(self):
        if self.value > 21:
            return True
        return False
    
    def isBlackjack(self):
        if len(self.cards) == 2:
            if self.value == 21:
                return True
        return False
        
    def calculateValue(self, prop = 'value'):
        value = sum(card.value for card in self.cards)
        aces = sum(card.isAce for card in self.cards)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        if (prop == 'aces'):
            return aces
        return value
        
    @property
    def value(self):
        return self.calculateValue()
    
    @property
    def soft(self):
        if self.calculateValue('aces'):
            return True
        return False
              
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
        
    def __repr__(self):
        return str(self.cards)
    
    def  __hash__(self):
        return self.value
    
    def __eq__(self, other):
        if len(self.cards) != len(other.cards):
            return False
        
        for card in self.cards:
            if card not in other.cards:
                return False
        return True

#### Testing Card, Deck, and Hand

## Player and Dealer

In [ ]:
# Player class keeps:
    # money
    # hands
class Player:
    
    def __init__(self, money = 5000, bet = 50):
        self.money = money
        self.betAmount = bet
        self.bets = []
        self.hands = [Hand()]
        
    def makeBet(self, handNumber):
        if handNumber >= len(self.bets): # new hand gets a new bet
            self.bets.append(self.betAmount)
        else: # double the bet
            self.bets[handNumber] += self.betAmount
        # remove the bet amount from players money
        self.money -= self.betAmount
            
    def resetBets(self):
        self.bets = []
        
    # True if player has enough money to make a bet, False otherwise
    def hasMoney(self):
        return self.money >= self.betAmount
        
    def hit(self, handNumber, card):
        self.hands[handNumber].addCard(card)

    def doubleDown(self, handNumber, card):
        self.hit(handNumber, card)
        self.makeBet(handNumber)

    def split(self, handNumber, cards):
        newHand = Hand()
        # remove one card from the hand being split and add to newHand
        newHand.addCard(self.hands[handNumber].removeCard())
        # add a second card to the old hand
        self.hands[handNumber].addCard(cards[0])  
        # add a second card to the new hand
        newHand.addCard(cards[1])
        # append newHand to players Hands and add bet for the hand
        self.hands.append(newHand)
        self.makeBet(len(self.hands)) # make a new bet for the new hand
        
        
    def showHands(self):
        print("Player has %d hands: \n" % len(self.hands))
        for hand in self.hands:
            print(hand)
        print() 
        
    def __str__(self):
        return str(self.hands)

# Dealer class keeps:
    # deck
    # hand
    # isBust
class Dealer:
    
    def __init__(self, deck, Player):
        self.deck = deck
        self.hand = Hand()
        self.player = Player
    
    def deal(self):
        for i in range(2): # deal each player, including the dealer, 2 cards
            self.hand.addCard(self.deck.draw()) # Deal the dealer a card
            self.player.hands[0].addCard(self.deck.draw()) # Deal the player a card   
        self.player.makeBet(0) # player makes a bet on hand
       
    def play(self):
        while self.hand.value <= 17: # hits until they have 17
            # if soft 17, dealer must hit
            if self.hand.value == 17:
                if self.hand.soft:
                    self.hit()
                else:
                    break
            # less than 17, must hit
            else:
                self.hit()
                      
    def hit(self):
        self.hand.addCard(self.deck.draw())
            
    def stand(self):
        return self.hand
    
    def showHands(self):
        print('Dealers Hand:')
        print(self.hand)

#### Test Player and Dealer

## Valid Moves and Make

In [ ]:
# returns a list of valid moves for a hand
def validMoves(player, handNumber):
    moves = ['hit','stand']
    # check that hand has 2 cards and the player has money to make a bet
    if len(player.hands[handNumber].cards) == 2:
        if player.hasMoney(): 
            moves.append('doubleDown')
            if player.hands[handNumber].cards[0].rank == player.hands[handNumber].cards[1].rank: # compare ranks same
                moves.append('split')
            
    return moves
    
def makeMove(game, handNumber, move):
    if move == 'hit':
        game.player.hit(handNumber, game.dealer.deck.draw())
        return False
    elif move == 'split':
        game.player.split(handNumber, [game.dealer.deck.draw(), game.dealer.deck.draw()])
        return False
    elif move == 'doubleDown':
        game.player.doubleDown(handNumber, game.dealer.deck.draw())
        return True
    else:
        return True



#### Testing ValidMoves and MakeMove

## Blackjack Game

In [89]:
class Blackjack:
    
    def __init__(self, player, deckSize=6):
        self.player = player
        self.deck = Deck(deckSize)
        self.deckSize = deckSize
        self.dealer = Dealer(self.deck, player) # Remember, dealer holds the deck and deals, not the game
        self.dealer.deal()
        
    def newHand(self):
        self.player.hands = [Hand()]
        self.player.resetBets()
        self.dealer.hand = Hand()
        self.dealer.deal()
        
    def newDeck(self):
        self.dealer.deck = Deck(self.deckSize)
       
    # hand is player's hand
    def gameStatus(self, hand):
        # player bust
        if hand.isBust():
            return -1
        # dealer bust
        if self.dealer.hand.isBust():
            return 1
        # draw
        if hand.value == self.dealer.hand.value:
            return 0
        # player win
        if hand.value > self.dealer.hand.value:
            return 1
        # dealer win
        if hand.value < self.dealer.hand.value:
            return -1
        
        

#### Test Blackjack

## TrainQ

In [422]:
import random
import numpy as np
import copy
import operator

# determines if a greedy move should be taken
def epsilonGreedy (epsilon, Q, player, handNumber, dealerCard, validMovesF):
    validMoves = validMovesF(player, handNumber)
    
    if np.random.uniform() < epsilon: # Random choice
        return random.choice(validMoves)
    else: # Greedy choice
        # Greedy Move
        Qs = np.array([Q.get(stateTuple(player.hands[handNumber], dealerCard, m), 0) for m in validMoves])
        return validMoves[ np.argmax(Qs) ]

def sortHand(hand):
    sortedHand = [card.rank for card in hand.cards]
    sortedHand.sort()
    return sortedHand
    
    
def stateTuple(hand, dealerCard, move):
    sortedHand = sortHand(hand)
    return (tuple(sortedHand), str(dealerCard), move)

def updateQ(Q, movesMade, value, learningRate):
    for i, state in enumerate(movesMade):
        # If new move update Q table with 0
        if state not in Q:
            Q[state] = 0
        elif i < len(movesMade)-1:
            # Update the move with learning rate
            Q[state] += learningRate*value
        else:
            Q[state] += value
    return Q

In [427]:

def trainQ(nRepetitions, learningRate, epsilonDecayRate, validMovesF, makeMoveF):
    epsilon = 1.0
    
    Q = {}
    outcomes = np.zeros(nRepetitions)
    epsilons = np.zeros(nRepetitions)
    
    for gameNum in range(nRepetitions):
        if gameNum%100 == 0:
            print("Game #%d" %(gameNum))
        epsilon *= epsilonDecayRate  # decay epsilon to move away from random choices
        epsilons[gameNum] = epsilon
        
        # create a game
        player = Player()
        blackjack = Blackjack(player)
        
        done = False
        
        # play some blackjack
        while len(blackjack.deck) >  52: # deck is 6 decks, last deck is a buffer deck
            if blackjack.player.hasMoney(): # make sure player has the funds to play
                blackjack.newHand() # deal a hand
                handNumber = 0
                handsToRemove = []
            
                # For each players hand
                while handNumber < len(blackjack.player.hands):
                    done = False

                    # Check for a blackjack
                    if blackjack.player.hands[handNumber].isBlackjack():
                        blackjack.player.money += .5*blackjack.player.bets[handNumber]
                        handsToRemove.append(handNumber)
                        done = True
                    
                    movesMade = [] # holds the moves made on this hand
                    step = 0
                    
                    # Player plays each hand until they stand or bust
                    while not done:
                        step += 1
                        
                        # Determine a move for the players hand
                        move = epsilonGreedy(epsilon, Q, blackjack.player, handNumber, 
                                             blackjack.dealer.hand.cards[1], validMovesF)
                        
                        newGame = copy.deepcopy(blackjack)
                        done = makeMoveF(newGame, handNumber, move)
                        
                        # if new move, add to Q
                        if stateTuple(blackjack.player.hands[handNumber], blackjack.dealer.hand.cards[1], move) not in Q:
                            Q[stateTuple(blackjack.player.hands[handNumber], blackjack.dealer.hand.cards[1], move)] = 0                        
                        
                        # if move results in 21, set Q value = to 0
                        if newGame.player.hands[handNumber].value == 21:
                            done = True
                        # if move results in bust, update Q with reinforcement
                        elif newGame.player.hands[handNumber].isBust():
                            done = True
        
                        # store a tuple (hand, dealercard, move) for later, player can only 'see' one of dealers cards
                        movesMade.append(stateTuple(blackjack.player.hands[handNumber], 
                                                    blackjack.dealer.hand.cards[1], move))
                        
                        blackjack = newGame
                            
                    handNumber += 1 # next hand
                
                numHands = len(handsToRemove)
                for i in range(numHands):
                    if i != 0:
                        handsToRemove[i] -= 1*i
                    del blackjack.player.hands[handsToRemove[i]]
                    del blackjack.player.bets[handsToRemove[i]]
                
                # After Player plays all his hands dealer plays
                blackjack.dealer.play()
                #print('moves: ' + str(movesMade))
                
                # After Dealer plays, check all players hands and payout on wins
                for hand,nothing in enumerate (blackjack.player.hands):
                    handStatus = blackjack.gameStatus(blackjack.player.hands[hand])
                    # update Q table
                    Q = updateQ(Q, movesMade, handStatus, learningRate)
                    if handStatus == 0: # tie (push)
                        blackjack.player.money += blackjack.player.bets[hand] # player gets their money back
                    if handStatus == 1: # win
                        blackjack.player.money += 2*blackjack.player.bets[hand] # player gets their money plus winnings
                        
            else:
                break # player doesn't have the funds, game over 
        # update outcome
        outcomes[gameNum] = blackjack.player.money
    return Q, outcomes, epsilons
        
    
    # test trainQ
    # sort hands before tuple
    # update Q - only last one
 
    
def testQ(Q, maxSteps , validMovesF, makeMoveF):
    
    player = Player()
    blackjack = Blackjack(player, 2)
    
    numberWins, numberBlackJacks, totalHands, numberPosWins, numberNegWins, numberZeroWins, numberLoses, numbers = playBlackjack(Q, blackjack, maxSteps, validMovesF, makeMoveF)
        
    return (str("Player Money: %d" %(blackjack.player.money)), 
            str("Player Winnings: %d" %(blackjack.player.money - 5000)), 
            str("Number of hands that were Blackjacks: %d" %(numberBlackJacks)),
            str("Win Percentage: %d, totalHands: %d" %(numberWins/totalHands*100, totalHands)),
            str("Number of Wins with Pos Reinforcement: %d" %(numberPosWins)),
            str("Number of Wins with Neg Reinforcement: %d" %(numberNegWins)),
            str("Number of Wins with 0 Reinforcement: %d" %(numberZeroWins)),
            str("Number of Pos, Neg, and Zero States: %s" %(numbers)),
            str("Number of loses: %d" %(numberLoses)))
        
def playBlackjack(Q, blackjack, numberHands, validMovesF, makeMoveF):
    numberWins = totalHands = numberPos = numberNeg = numberZero = numberPosWins = numberNegWins = 0 
    numberLoses = numberZeroWins = numberBlackJacks = 0
    wonPos = []
    for i in range(numberHands):
        if not blackjack.player.hasMoney():
            break

        if len(blackjack.dealer.deck) < 52:
            blackjack.newDeck()
            
        blackjack.newHand() # deal a hand
        handNumber = 0
        handsToRemove = []
        while handNumber < len(blackjack.player.hands):
            done = False
            totalHands += 1
            
            if blackjack.player.hands[handNumber].isBlackjack():
                blackjack.player.money += 2*blackjack.player.bets[handNumber] + .5*blackjack.player.bets[handNumber]
                handsToRemove.append(handNumber)
                numberBlackJacks += 1
                numberWins += 0
                done = True
            
            while not done: 
                validMoves = validMovesF(blackjack.player, handNumber)
                Qs = np.array([Q.get(stateTuple(blackjack.player.hands[handNumber],
                                                blackjack.dealer.hand.cards[1], m), 0) for m in validMoves])
                move = validMoves[ np.argmax(Qs) ] # choose move
                done = makeMoveF(blackjack, handNumber, move)
                
                if Qs[np.argmax(Qs)] > 0:
                    numberPos += 1
                elif Qs[np.argmax(Qs)] == 0:
                    numberZero += 1
                else:
                    numberNeg += 1
                
                if blackjack.player.hands[handNumber].value == 21:
                    done = True
                if blackjack.player.hands[handNumber].isBust():
                    done = True
                
                if done:
                    .append(Qs[np.argmax(Qs)])
                   
            handNumber += 1
          
        
        numHands = len(handsToRemove)
        for i in range(numHands):
            if i != 0:
                handsToRemove[i] -= 1*i
            del blackjack.player.hands[handsToRemove[i]]
            del blackjack.player.bets[handsToRemove[i]]
                
        blackjack.dealer.play()
        
        # total hands
        for hand,nothing in enumerate (blackjack.player.hands):
            handStatus = blackjack.gameStatus(blackjack.player.hands[hand])
            if handStatus == 0: # tie (push)
                blackjack.player.money += blackjack.player.bets[hand] # player gets their money backts their money back
            if handStatus == 1: # win
                if states[hand] > 0:
                    numberPosWins += 1
                if states[hand] < 0:
                    numberNegWins += 1
                if states[hand] == 0:
                    numberZeroWins += 1
                numberWins += 1
                blackjack.player.money += 2*blackjack.player.bets[hand] # player gets their money plus winnings
            else:
                numberLoses += 1
    return numberWins, numberBlackJacks, totalHands, numberPosWins, numberNegWins, numberZeroWins, numberLoses, [numberPos, -numberNeg, numberZero]

## Testing

In [ ]:

Q,outcomes,_ = trainQ(20000, 0.7, 0.99998, validMoves, makeMove)

print(outcomes)

Game #0
Game #100
Game #200
Game #300
Game #400
Game #500
Game #600
Game #700
Game #800
Game #900
Game #1000
Game #1100
Game #1200
Game #1300
Game #1400
Game #1500
Game #1600
Game #1700


In [429]:
print(Q)

{(('2', 'A', 'Q'), 'A', 'doubleDown'): -10, (('10', '10', '6'), 'Q', 'doubleDown'): -31, (('10', 'K'), '7', 'stand'): 267, (('7', 'J'), '9', 'stand'): -121, (('10', '8', 'Q'), '6', 'hit'): -51, (('3', '4', '9'), 'Q', 'hit'): -38.50000000000001, (('3', '4', '7', '9'), 'Q', 'hit'): -8, (('10', '6', '8'), 'A', 'hit'): -95, (('6', '7', '9'), '8', 'hit'): -90, (('2', '7', '9'), '3', 'doubleDown'): 5, (('3', '7', 'K'), 'J', 'hit'): 9.799999999999999, (('3', '7', 'K'), 'J', 'stand'): 36, (('7', '9', 'K'), 'J', 'doubleDown'): -54, (('10', '8', 'J'), '6', 'doubleDown'): -57, (('3', '6'), '9', 'stand'): -90, (('8', 'K', 'Q'), 'Q', 'hit'): -77, (('10', '3', 'J'), '4', 'hit'): -76, (('5', '6', '7'), 'A', 'hit'): -45.500000000000036, (('5', '6', '7'), 'A', 'stand'): -28, (('2', '3', 'K'), '6', 'doubleDown'): -21, (('4', '6', '7'), '6', 'hit'): -20.99999999999999, (('4', '6', '7', '8'), '6', 'hit'): -9, (('3', '4', '9'), '9', 'doubleDown'): -39, (('2', '4'), '2', 'stand'): -67, (('2', '3', 'J'), 'Q'

In [437]:
gameStats = testQ(Q, 1000 , validMoves, makeMove)

for stat in gameStats:
    print(stat)

Player Money: 25
Player Winnings: -4975
Number of hands that were Blackjacks: 43
Win Percentage: 32, totalHands: 793
Number of Wins with Pos Reinforcement: 0
Number of Wins with Neg Reinforcement: 258
Number of Wins with 0 Reinforcement: 0
Number of Pos, Neg, and Zero States: [260, -238, 613]
Number of loses: 492
